In [1]:
!pip install opendatasets



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import opendatasets as od
od.download("https://www.kaggle.com/competitions/image-processing-thai-language-image-captioning")
     

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  tuchsanai


Your Kaggle Key:

  ········


100%|██████████| 1.75G/1.75G [01:25<00:00, 22.0MB/s]



Extracting archive ./image-processing-thai-language-image-captioning/image-processing-thai-language-image-captioning.zip to ./image-processing-thai-language-image-captioning


  ········


100%|██████████| 1.75G/1.75G [00:41<00:00, 45.5MB/s]



Extracting archive ./image-processing-thai-language-image-captioning/image-processing-thai-language-image-captioning.zip to ./image-processing-thai-language-image-captioning


# Train

In [6]:
import os

# Define the folder paths
food_folder_path = "./image-processing-thai-language-image-captioning/train/train/food"
travel_folder_path = "./image-processing-thai-language-image-captioning/train/train/travel"

# List of valid image extensions
image_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp")

# List all image files in the food folder
food_image_files = [
    f for f in os.listdir(food_folder_path) 
    if f.lower().endswith(image_extensions)
]
# List all image files in the travel folder
travel_image_files = [
    f for f in os.listdir(travel_folder_path) 
    if f.lower().endswith(image_extensions)
]

# Convert filenames to full paths
food_image_paths = [os.path.join(food_folder_path, f) for f in food_image_files]
travel_image_paths = [os.path.join(travel_folder_path, f) for f in travel_image_files]

# Combine both into one list
train_image_paths = food_image_paths + travel_image_paths

print("Number of food images =", len(food_image_files))
print("Number of travel images =", len(travel_image_files))
print("Total images in train folder =", len(train_image_paths))


Number of food images = 11693
Number of travel images = 16311
Total images in train folder = 28004


In [7]:
import json 
from tqdm import tqdm

json_file = "./image-processing-thai-language-image-captioning/capgen_v1.0_train.json"

# STEP 1: Load the JSON
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

jdata = list(data.keys())

L =[]

for x in tqdm(train_image_paths) :
   y = x.split("/")[-1] 
   for j in jdata :
     jj=j.split("/")[-1]
     if jj ==y :  
        L.append({"image_path":x,"captions":data[j]})  

100%|██████████| 28004/28004 [15:41<00:00, 29.74it/s]


In [8]:
from datasets import Dataset
from datasets import Dataset
from PIL import Image


def load_image(example):
    # Open the image from the given file path
    image = Image.open(example["image_path"])
    # Return a dictionary mapping the "image" key to the loaded image object
    return {"image": image}


# data_for_hf is the list of dictionaries created above
train_dataset = Dataset.from_list(L)
train_dataset = train_dataset.map(load_image,remove_columns=["image_path"])

train_dataset


Map:   0%|          | 0/28004 [00:00<?, ? examples/s]

Dataset({
    features: ['captions', 'image'],
    num_rows: 28004
})

# Val

In [10]:
import os

# Define the folder paths for validation
valfood_folder_path = "./image-processing-thai-language-image-captioning/val/val/food"
valtravel_folder_path = "./image-processing-thai-language-image-captioning/val/val/travel"

# List of valid image extensions
image_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp")

# List all image files in the validation food folder
val_food_image_files = [
    f for f in os.listdir(valfood_folder_path) 
    if f.lower().endswith(image_extensions)
]

# List all image files in the validation travel folder
val_travel_image_files = [
    f for f in os.listdir(valtravel_folder_path) 
    if f.lower().endswith(image_extensions)
]

# Convert filenames to full paths
val_food_image_paths = [
    os.path.join(valfood_folder_path, f) for f in val_food_image_files
]
val_travel_image_paths = [
    os.path.join(valtravel_folder_path, f) for f in val_travel_image_files
]

# Combine both into one list
val_image_paths = val_food_image_paths + val_travel_image_paths

# Print summary
print("Number of food images in val folder =", len(val_food_image_files))
print("Number of travel images in val folder =", len(val_travel_image_files))
print("Total images in val folder =", len(val_image_paths))


Number of food images in val folder = 1668
Number of travel images in val folder = 2368
Total images in val folder = 4036


In [11]:
import json 
from tqdm import tqdm


json_file = "./image-processing-thai-language-image-captioning/capgen_v1.0_val.json"

# STEP 1: Load the JSON
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

jdata = list(data.keys())


L =[]

for x in tqdm(val_image_paths) :
   y = x.split("/")[-1] 
   for j in jdata :
     jj=j.split("/")[-1]
     if jj ==y :  
        L.append({"image_path":x,"captions":data[j]})  

100%|██████████| 4036/4036 [00:08<00:00, 452.22it/s]


In [12]:
from datasets import Dataset


def load_image(example):
    # Open the image from the given file path
    image = Image.open(example["image_path"])
    # Return a dictionary mapping the "image" key to the loaded image object
    return {"image": image}

# data_for_hf is the list of dictionaries created above
val_dataset = Dataset.from_list(L)
val_dataset = val_dataset.map(load_image,remove_columns=["image_path"])
val_dataset 

Map:   0%|          | 0/4036 [00:00<?, ? examples/s]

Dataset({
    features: ['captions', 'image'],
    num_rows: 4036
})

# Combine

In [13]:
from datasets import concatenate_datasets, load_dataset

combined_dataset = concatenate_datasets([train_dataset, val_dataset ])
combined_dataset = combined_dataset.shuffle(seed=42)

combined_dataset

Dataset({
    features: ['captions', 'image'],
    num_rows: 32040
})

In [15]:
from datasets import load_dataset



# Push the dataset to your Hugging Face Hub repository
combined_dataset.push_to_hub("xxx", token="xxxxx")


Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ?it/s]

Map:   0%|          | 0/8010 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/81 [00:00<?, ?ba/s]

Map:   0%|          | 0/8010 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/81 [00:00<?, ?ba/s]

Map:   0%|          | 0/8010 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/81 [00:00<?, ?ba/s]

Map:   0%|          | 0/8010 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/81 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Tuch/food_and_travel/commit/3f588e14d4f40c622d4a4a91f7abf8718556cf3c', commit_message='Upload dataset', commit_description='', oid='3f588e14d4f40c622d4a4a91f7abf8718556cf3c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Tuch/food_and_travel', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Tuch/food_and_travel'), pr_revision=None, pr_num=None)